In [1]:
import pandas as pd
import re
import ftfy
import spacy
from spacy import displacy
from tqdm import tqdm

In [2]:
df = pd.read_csv("data/processed/burndata.csv")
df

,source,target,year,genre,source_location,text
0,burn,anger,1991,ACAD,Symposium,a meaning to this manifestation. I turned to t...
1,burn,desire,1997,ACAD,HispanicRev,male) lover rather than the (female) beloved. ...
2,burn,desire,1996,ACAD,AmerIndianQ,by Pueblos for whom making such knowledge publ...
3,burn,desire,1995,ACAD,Symposium,"at the same time requires his sperm: a "" famis..."
4,burn,passion,2012,ACAD,JournalAmerican,? - and did the dust Of these fair solitudes o...
...,...,...,...,...,...,...
307,burn,anguish,2012,WEB,mendele.com,I saw myself as a creature driven and derided ...
308,burn,ecstasy,2012,WEB,goodreads.com,will one day discard but for his inner being. ...
309,burn,gratitude,2012,WEB,bible.cc,one of the humblest conditions of the children...
310,burn,desire,2012,WEB,goodreads.com,"fast for Rocco, never giving up on him... # A ..."


In [165]:
fire = pd.read_csv("data/raw/BURN_PREP_FIRE_DATA.csv")
fire = fire[["source", "target", "index", "year", "genre", "source.1", "text"]]
fire
fire['clean_text'] = fire['text'].apply(clean_text)
fire.to_csv("data/processed/firedata_nometaphor.csv")


In [200]:
emo = pd.read_csv("data/raw/HEAT_PREP_EMO_DATA.csv")
emo = emo[["source", "target", "index", "year", "genre", "source.1", "text"]]
emo
emo['clean_text'] = emo['text'].apply(clean_text)
emo.to_csv("data/processed/emodata.csv")


In [3]:


def clean_text(s):
    if pd.isna(s):
        return ""
    s = ftfy.fix_text(str(s))                # fix unicode mojibake
    s = s.replace("\xa0"," ")                # non-breaking spaces
    s = re.sub(r"[\r\t]+", " ", s)           # remove stray control chars
    s = re.sub(r"\s+", " ", s).strip()       # collapse whitespace
    return s

df['clean_text'] = df['text'].apply(clean_text)


In [4]:
nlp = spacy.load("en_core_web_lg")

C:\Users\micah\PycharmProjects\computational_linguistics\.venv\Lib\site-packages\spacy\util.py:894: UserWarning: [W095] Model 'en_core_web_lg' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (4.0.0.dev3). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
C:\Users\micah\PycharmProjects\computational_linguistics\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
texts = df['clean_text'].copy()

In [13]:
docs = []
for text in texts:
    doc = nlp(text)
    docs.append(doc)

In [14]:
test = docs[:5]

for i in test:
    for j in i:
        print(j, j.sentiment)

AttributeError: 'spacy.tokens.token.Token' object has no attribute 'sentiment'

In [170]:
texts_fire = fire['clean_text'].copy()
texts_fire_df = pd.DataFrame(texts_fire)


texts_fire_df.columns = ["articles"]  # ensures correct column name

articles_json = {
    "Articles": texts_fire_df["articles"].tolist()
}
import json

with open("fire_articles.json", "w", encoding="utf-8") as f:
    json.dump(articles_json, f, ensure_ascii=False, indent=2)

In [201]:
texts_emo = emo['clean_text'].copy()
texts_emo_df = pd.DataFrame(texts_emo)


texts_emo_df.columns = ["articles"]  # ensures correct column name

articles_json = {
    "articles": texts_emo_df["articles"].tolist()
}
import json

with open("emo_articles.json", "w", encoding="utf-8") as f:
    json.dump(articles_json, f, ensure_ascii=False, indent=2)

In [16]:
texts

0      a meaning to this manifestation. I turned to t...
1      male) lover rather than the (female) beloved. ...
2      by Pueblos for whom making such knowledge publ...
3      at the same time requires his sperm: a " famis...
4      ? - and did the dust Of these fair solitudes o...
                             ...                        
307    I saw myself as a creature driven and derided ...
308    will one day discard but for his inner being. ...
309    one of the humblest conditions of the children...
310    fast for Rocco, never giving up on him... # A ...
311    upper echelons. He would have, too, if he didn...
Name: clean_text, Length: 312, dtype: object

In [20]:
textdf = pd.DataFrame(texts)
textdf

,clean_text
0,a meaning to this manifestation. I turned to t...
1,male) lover rather than the (female) beloved. ...
2,by Pueblos for whom making such knowledge publ...
3,"at the same time requires his sperm: a "" famis..."
4,? - and did the dust Of these fair solitudes o...
...,...
307,I saw myself as a creature driven and derided ...
308,will one day discard but for his inner being. ...
309,one of the humblest conditions of the children...
310,"fast for Rocco, never giving up on him... # A ..."


In [24]:
textdf.columns = ["Articles"]     # ensures correct column name

articles_json = {
    "Articles": textdf["Articles"].tolist()
}
import json
with open("articles.json", "w", encoding="utf-8") as f:
    json.dump(articles_json, f, ensure_ascii=False, indent=2)


In [30]:
framebert_df = pd.read_csv("predictions.tsv", sep="\t")

In [171]:
fire_framebert_df = pd.read_csv("literal_predictions.tsv", sep="\t")

ffb = fire_framebert_df

ffb

,Article,Sentence,Tokens,Borderline_metaphor,Real_metaphors,Frame_label
0,0,0,.,0,0,_
1,0,1,Illusions,1,1,Medical_conditions
2,0,1,fed,1,1,Supply
3,0,1,",",0,0,_
4,0,1,lo,0,0,_
...,...,...,...,...,...,...
2363,76,0,to,0,0,_
2364,76,0,win,0,0,Finish_competition
2365,76,0,journalism,0,0,_
2366,76,0,'s,0,0,_


In [202]:
emo_df = pd.read_csv("emo_predictions.tsv", sep="\t")

e = emo_df

e

,Article,Sentence,Tokens,Borderline_metaphor,Real_metaphors,Frame_label
0,0,0,was,0,0,_
1,0,0,tormented,1,1,Emotion_directed
2,0,0,with,1,0,_
3,0,0,his,0,0,_
4,0,0,life,0,0,_
...,...,...,...,...,...,...
10284,333,2,of,0,0,Partitive
10285,333,2,your,0,0,_
10286,333,2,body,0,0,Body_parts
10287,333,2,.,0,0,_


In [35]:
framebert_df

,Article,Sentence,Tokens,Borderline_metaphor,Real_metaphors,Frame_label
0,0,0,a,0,0,_
1,0,0,meaning,0,0,_
2,0,0,to,1,0,_
3,0,0,this,1,0,_
4,0,0,manifestation,0,0,Sign
...,...,...,...,...,...,...
9576,311,1,.,0,0,_
9577,311,2,#,0,0,_
9578,311,2,Though,0,0,Concessive
9579,311,2,still,0,0,Continued_state_of_affairs


In [65]:
framebert_df.groupby("Article")["Frame_label"].value_counts()

ValueError: 'Article' is both an index level and a column label, which is ambiguous.

In [37]:
framebert_df.Frame_label.value_counts()

Frame_label
_                     6426
Fire_burning           300
Experiencer_focus      217
Body_parts             184
Emotion_directed       124
                      ... 
Being_at_risk            1
Fluidic_motion           1
Temperature              1
Opportunity              1
Member_of_military       1
Name: count, Length: 373, dtype: int64

In [206]:
def safe_lemmatize(token):
    if isinstance(token, str):
        return nlp(token)[0].lemma_
    else:
        return token

framebert_df['Token_lemma'] = framebert_df['Tokens'].apply(safe_lemmatize)
ffb['Token_lemma'] = ffb['Tokens'].apply(safe_lemmatize)
e['Token_lemma'] = e['Tokens'].apply(safe_lemmatize)


In [175]:
def distance_from_lemma(group):
    indices = group.index[group['is_target']].tolist()
    if not indices:
        group['distance_from_target'] = None
        return group
    first_index = indices[0]
    group['distance_from_target'] = (group.index.to_numpy() - first_index).astype(int).tolist()
    return group

target_lemma = "burn"
# framebert_df['is_target'] = framebert_df['Token_lemma'] == target_lemma
#
# framebert_df = framebert_df.groupby("Article").apply(distance_from_lemma)
# framebert_df = framebert_df.rename(columns={"Article": "Article_col"})

ffb['is_target'] = ffb['Token_lemma'] == target_lemma
ffb = ffb.groupby("Article").apply(distance_from_lemma)
ffb = ffb.rename(columns={"Article": "Article_col"})

"""TODO need to figure out how to make it work with non-burn targets"""


C:\Users\micah\AppData\Local\Temp\ipykernel_8608\3075259160.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ffb = ffb.groupby("Article").apply(distance_from_lemma)


In [52]:
framebert_df

Article  Sentence         Tokens  Borderline_metaphor  \
Article                                                               
0       0           0         0              a                    0   
        1           0         0        meaning                    0   
        2           0         0             to                    1   
        3           0         0           this                    1   
        4           0         0  manifestation                    0   
...               ...       ...            ...                  ...   
311     9576      311         1              .                    0   
        9577      311         2              #                    0   
        9578      311         2         Though                    0   
        9579      311         2          still                    0   
        9580      311         2       commonly                    0   

              Real_metaphors                 Frame_label    Token_lemma  \
Article                                                                   
0       0                  0                           _              a   
        1                  0                           _           mean   
        2                  0                           _             to   
        3                  0                           _           this   
        4                  0                        Sign  manifestation   
...                      ...                         ...            ...   
311     9576               0                           _              .   
        9577               0                           _              #   
        9578               0                  Concessive         though   
        9579               0  Continued_state_of_affairs          still   
        9580               0                           _       commonly   

              is_target distance_from_target  
Article                                       
0       0         False                  -15  
        1         False                  -14  
        2         False                  -13  
        3         False                  -12  
        4         False                  -11  
...                 ...                  ...  
311     9576      False                   11  
        9577      False                   12  
        9578      False                   13  
        9579      False                   14  
        9580      False                   15  

[9581 rows x 9 columns]

In [53]:
framebert_df.groupby("Frame_label")["distance_from_target"].mean()

Frame_label
Abandonment             -7.5
Abounding_with         0.875
Accompaniment       2.166667
Accoutrements      -8.142857
Activity_ongoing         2.8
                      ...   
Wearing                -14.0
Weather                 -4.0
Willingness             13.0
Work                    10.0
_                   0.404863
Name: distance_from_target, Length: 373, dtype: object

In [60]:
framebert_df[framebert_df["Borderline_metaphor"] == 1].distance_from_target.mean()

0.5414908579465542

In [61]:
framebert_df[framebert_df["Real_metaphors"] == 1].distance_from_target.mean()

0.2926829268292683

In [73]:
avg_distance_borderline_frame = (
    framebert_df[framebert_df['Borderline_metaphor'] == 1]
    .groupby('Frame_label')['distance_from_target']
    .mean()
)

avg_distance_borderline_frame

Frame_label
Abounding_with                  5.0
Activity_resume                -8.0
Activity_stop                 -14.0
Adopt_selection               -13.0
Aesthetics                     13.0
                             ...   
Transition_to_a_quality        13.0
Transition_to_state       -5.333333
Turning_out                   -13.5
Type                           13.0
_                          1.055409
Name: distance_from_target, Length: 150, dtype: object

In [71]:
avg_distance_real_frame = (
    framebert_df[framebert_df['Real_metaphors'] == 1]
    .groupby('Frame_label')['distance_from_target']
    .mean()
)

avg_distance_real_frame

Frame_label
Abounding_with       1.5
Activity_resume     -8.0
Adopt_selection    -13.0
Aesthetics          13.0
Assessing            6.0
                   ...  
Taking_time         -1.0
Theft               10.0
Topic              -11.0
Toxic_substance     13.0
_                  0.864
Name: distance_from_target, Length: 69, dtype: object

In [80]:
avg_distance_nonmet_frame = (
    framebert_df[(framebert_df['Borderline_metaphor'] == 0) & (framebert_df['Real_metaphors'] == 0)]
    .groupby('Frame_label')['distance_from_target']
    .mean()
)

avg_distance_nonmet_frame

Frame_label
Abandonment             -7.5
Abounding_with         -3.25
Accompaniment       2.166667
Accoutrements      -8.142857
Activity_ongoing         2.8
                      ...   
Wearing                -14.0
Weather                 -4.0
Willingness             13.0
Work                    10.0
_                   0.317073
Name: distance_from_target, Length: 339, dtype: object

In [74]:
framebert_df[framebert_df['distance_from_target'] == 2].Frame_label.value_counts()

Frame_label
Experiencer_focus          135
Emotion_directed            83
_                           41
Desiring                    24
Judgment                     3
Needing                      3
Certainty                    3
Fear                         3
Personal_relationship        2
Medical_conditions           2
Reasoning                    1
Body_parts                   1
Fire_burning                 1
Awareness                    1
Expectation                  1
Likelihood                   1
Opinion                      1
Importance                   1
Purpose                      1
Becoming_aware               1
Judgment_direct_address      1
Name: count, dtype: int64

In [78]:
framebert_df[(framebert_df.Frame_label == "Experiencer_focus") & (framebert_df.distance_from_target == 2)].Token_lemma.value_counts()

Token_lemma
shame            29
embarrassment    23
jealousy         10
hatred           10
love              8
curiosity         7
humiliation       7
hate              6
lust              5
envy              4
guilt             3
resentment        3
passion           2
pride             2
compassion        2
love/             2
horror            1
nervousness       1
pity              1
discomfort        1
ardor             1
satisfaction      1
regret            1
relish            1
impatience        1
agony             1
animosity         1
ecstasy           1
Name: count, dtype: int64

In [79]:
framebert_df[(framebert_df.Frame_label == "Emotion_directed") & (framebert_df.distance_from_target == 2)].Token_lemma.value_counts()


Token_lemma
anger             24
rage              12
indignation        7
fury               7
excitement         5
frustration        4
shame              4
impatience         3
worry              3
anguish            3
annoyance          2
grief              2
agony              1
disappointment     1
anxiety            1
outrage            1
sorrow             1
surprise           1
irritation         1
Name: count, dtype: int64

In [82]:
framebert_df

Article_col  Sentence         Tokens  Borderline_metaphor  \
Article                                                                   
0       0               0         0              a                    0   
        1               0         0        meaning                    0   
        2               0         0             to                    1   
        3               0         0           this                    1   
        4               0         0  manifestation                    0   
...                   ...       ...            ...                  ...   
311     9576          311         1              .                    0   
        9577          311         2              #                    0   
        9578          311         2         Though                    0   
        9579          311         2          still                    0   
        9580          311         2       commonly                    0   

              Real_metaphors                 Frame_label    Token_lemma  \
Article                                                                   
0       0                  0                           _              a   
        1                  0                           _           mean   
        2                  0                           _             to   
        3                  0                           _           this   
        4                  0                        Sign  manifestation   
...                      ...                         ...            ...   
311     9576               0                           _              .   
        9577               0                           _              #   
        9578               0                  Concessive         though   
        9579               0  Continued_state_of_affairs          still   
        9580               0                           _       commonly   

              is_target distance_from_target  
Article                                       
0       0         False                  -15  
        1         False                  -14  
        2         False                  -13  
        3         False                  -12  
        4         False                  -11  
...                 ...                  ...  
311     9576      False                   11  
        9577      False                   12  
        9578      False                   13  
        9579      False                   14  
        9580      False                   15  

[9581 rows x 9 columns]

In [87]:
framebert_df[(framebert_df["Real_metaphors"] == 1) & (framebert_df["distance_from_target"] != 0) & (framebert_df["distance_from_target"] != 2)]

Article_col  Sentence     Tokens  Borderline_metaphor  \
Article                                                               
3       103             3         0   famished                    1   
        113             3         0     hollow                    1   
        115             3         0       lazy                    1   
        123             3         0  devouring                    1   
4       129             4         1       dust                    1   
...                   ...       ...        ...                  ...   
308     9485          308         1    central                    1   
309     9493          309         0   humblest                    1   
310     9521          310         0       fast                    1   
        9538          310         0    flashes                    1   
        9540          310         0    ignites                    1   

              Real_metaphors        Frame_label Token_lemma  is_target  \
Article                                                                  
3       103                1    Biological_urge    famished      False   
        113                1                  _      hollow      False   
        115                1                  _        lazy      False   
        123                1          Ingestion      devour      False   
4       129                1                  _        dust      False   
...                      ...                ...         ...        ...   
308     9485               1                  _     central      False   
309     9493               1                  _      humble      False   
310     9521               1  Speed_description        fast      False   
        9538               1    Moving_in_place       flash      False   
        9540               1       Setting_fire      ignite      False   

             distance_from_target  
Article                            
3       103                    -5  
        113                     5  
        115                     7  
        123                    15  
4       129                   -10  
...                           ...  
308     9485                   11  
309     9493                  -12  
310     9521                  -13  
        9538                    4  
        9540                    6  

[226 rows x 9 columns]

In [94]:
framebert_df[(framebert_df["Borderline_metaphor"] == 1) & (framebert_df["distance_from_target"] != 0) & (framebert_df["distance_from_target"] != 2)]


Article_col  Sentence   Tokens  Borderline_metaphor  \
Article                                                             
0       2               0         0       to                    1   
        3               0         0     this                    1   
        16              0         1     with                    1   
        22              0         1       to                    1   
1       47              1         1     with                    1   
...                   ...       ...      ...                  ...   
310     9538          310         0  flashes                    1   
        9540          310         0  ignites                    1   
        9541          310         0  between                    1   
311     9566          311         1     from                    1   
        9568          311         1     with                    1   

              Real_metaphors                Frame_label Token_lemma  \
Article                                                               
0       2                  0                          _          to   
        3                  0                          _        this   
        16                 0                          _        with   
        22                 0                          _          to   
1       47                 0                          _        with   
...                      ...                        ...         ...   
310     9538               1            Moving_in_place       flash   
        9540               1               Setting_fire      ignite   
        9541               0  Interior_profile_relation     between   
311     9566               0                          _        from   
        9568               0                          _        with   

              is_target distance_from_target  
Article                                       
0       2         False                  -13  
        3         False                  -12  
        16        False                    1  
        22        False                    7  
1       47        False                    1  
...                 ...                  ...  
310     9538      False                    4  
        9540      False                    6  
        9541      False                    7  
311     9566      False                    1  
        9568      False                    3  

[1121 rows x 9 columns]

In [98]:
fb = framebert_df


Article_col  Sentence     Tokens  Borderline_metaphor  \
Article                                                               
0       10              0         1     Polish                    0   
        11              0         1  violinist                    0   
        12              0         1          ,                    0   
        13              0         1        who                    0   
        14              0         1        was                    0   
...                   ...       ...        ...                  ...   
311     9560          311         1         he                    0   
        9561          311         1        did                    0   
        9562          311         1        n't                    0   
        9563          311         1      crash                    0   
        9564          311         1        and                    0   

              Real_metaphors Frame_label Token_lemma  is_target  \
Article                                                           
0       10                 0      Origin      polish      False   
        11                 0           _   violinist      False   
        12                 0           _           ,      False   
        13                 0           _         who      False   
        14                 0           _          be      False   
...                      ...         ...         ...        ...   
311     9560               0           _          he      False   
        9561               0           _          do      False   
        9562               0           _         not      False   
        9563               0           _       crash      False   
        9564               0           _         and      False   

             distance_from_target  
Article                            
0       10                     -5  
        11                     -4  
        12                     -3  
        13                     -2  
        14                     -1  
...                           ...  
311     9560                   -5  
        9561                   -4  
        9562                   -3  
        9563                   -2  
        9564                   -1  

[1539 rows x 9 columns]

In [102]:
range_tuples = [(30, 25), (25, 20), (20, 15), (15, 10), (10, 3), (0, -5), (-5, -10), (-10, -15), (-15, -20)]
for i in range_tuples:
    print(i)
    print(
        fb[(fb['distance_from_target'] < i[0]) & (fb['distance_from_target'] >= i[1])].Frame_label.value_counts()

    )
fb[(fb['distance_from_target'] < 0) & (fb['distance_from_target'] >= -5)]


(30, 25)
Frame_label
_                            13
Interior_profile_relation     1
Name: count, dtype: int64
(25, 20)
Frame_label
_                             19
Emotion_directed               2
Attempt                        1
Duration_description           1
Fire_burning                   1
Abounding_with                 1
Desiring                       1
Continued_state_of_affairs     1
Name: count, dtype: int64
(20, 15)
Frame_label
_                             230
Motion                          5
Perception_experience           4
Capability                      4
Experiencer_focus               4
Intentionally_act               3
Body_parts                      3
People                          3
Age                             2
Give_impression                 2
Self_motion                     2
Desirability                    2
Calendric_unit                  2
Temporal_collocation            2
Emotion_directed                2
Being_employed                  2
Memory       

Article_col  Sentence     Tokens  Borderline_metaphor  \
Article                                                               
0       10              0         1     Polish                    0   
        11              0         1  violinist                    0   
        12              0         1          ,                    0   
        13              0         1        who                    0   
        14              0         1        was                    0   
...                   ...       ...        ...                  ...   
311     9560          311         1         he                    0   
        9561          311         1        did                    0   
        9562          311         1        n't                    0   
        9563          311         1      crash                    0   
        9564          311         1        and                    0   

              Real_metaphors Frame_label Token_lemma  is_target  \
Article                                                           
0       10                 0      Origin      polish      False   
        11                 0           _   violinist      False   
        12                 0           _           ,      False   
        13                 0           _         who      False   
        14                 0           _          be      False   
...                      ...         ...         ...        ...   
311     9560               0           _          he      False   
        9561               0           _          do      False   
        9562               0           _         not      False   
        9563               0           _       crash      False   
        9564               0           _         and      False   

             distance_from_target  
Article                            
0       10                     -5  
        11                     -4  
        12                     -3  
        13                     -2  
        14                     -1  
...                           ...  
311     9560                   -5  
        9561                   -4  
        9562                   -3  
        9563                   -2  
        9564                   -1  

[1539 rows x 9 columns]

In [157]:
range_tuples = [(30, 22), (22, 17), (17, 12), (12, 7), (7, 3), (0, -5), (-5, -10), (-10, -15), (-15, -20)]
results = []

for start, end in range_tuples:
    subset = fb[(fb['distance_from_target'] < start) & (fb['distance_from_target'] >= end)]

    # per-article presence of metaphors
    has_metaphor = subset.groupby('Article_col')['Real_metaphors'].any()
    has_borderline = subset.groupby('Article_col')['Borderline_metaphor'].any()

    count_with_metaphor = has_metaphor.sum()
    count_with_borderline = has_borderline.sum()

    total = has_metaphor.shape[0]

    # rates
    rate_real = count_with_metaphor / total if total > 0 else 0
    rate_borderline = count_with_borderline / total if total > 0 else 0
    rate_total = (count_with_metaphor + count_with_borderline) / total if total > 0 else 0

    results.append({
        'interval': f"{start} to {end}",
        'count_with_metaphor': count_with_metaphor,
        'count_with_borderline': count_with_borderline,
        'total_sentences': total,
        'rate_real': rate_real,
        'rate_borderline': rate_borderline,
        'rate_total': rate_total
    })

df_results = pd.DataFrame(results)
df_results

,interval,count_with_metaphor,count_with_borderline,total_sentences,rate_real,rate_borderline,rate_total
0,30 to 22,2,3,5,0.400000,0.600000,1.000000
1,22 to 17,0,4,12,0.000000,0.333333,0.333333
2,17 to 12,32,90,309,0.103560,0.291262,0.394822
3,12 to 7,40,124,310,0.129032,0.400000,0.529032
4,7 to 3,32,100,310,0.103226,0.322581,0.425806
5,0 to -5,29,96,310,0.093548,0.309677,0.403226
6,-5 to -10,40,124,304,0.131579,0.407895,0.539474
7,-10 to -15,35,106,303,0.115512,0.349835,0.465347
8,-15 to -20,0,1,7,0.000000,0.142857,0.142857


In [197]:
import string
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

# Filter rows in your interval
subset = fb[(-5 <= fb.distance_from_target) & (fb.distance_from_target < 0)]

# Keep only tokens that are not stopwords and are alphabetic
filtered_tokens = subset[
    ~subset['Tokens'].str.lower().isin(stop_words) &
    subset['Tokens'].str.isalpha()
]['Tokens']

# Get value counts
filtered_tokens.value_counts()


Tokens
eyes        35
face        22
cheeks      16
felt         7
heart        6
            ..
skeleton     1
living       1
Alchylus     1
ready        1
crash        1
Name: count, Length: 423, dtype: int64

In [198]:
import string
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

# Filter rows in your interval
subset = fb[(3 <= fb.distance_from_target) & (fb.distance_from_target < 10)]

# Keep only tokens that are not stopwords and are alphabetic
filtered_tokens = subset[
    ~subset['Tokens'].str.lower().isin(stop_words) &
    subset['Tokens'].str.isalpha()
]['Tokens']

# Get value counts
filtered_tokens.value_counts()

Tokens
would      6
burns      6
love       5
one        5
anger      5
          ..
came       1
prayed     1
hoplite    1
Damned     1
ragged     1
Name: count, Length: 632, dtype: int64

In [137]:
fb[~fb.distance_from_target.isin([0, 1, 2])].groupby("Article_col")[["Borderline_metaphor", "Real_metaphors"]].value_counts()

Article_col  Borderline_metaphor  Real_metaphors
0            0                    0                 25
             1                    0                  3
1            0                    0                 27
             1                    0                  1
2            0                    0                 27
                                                    ..
310          0                    0                 20
             1                    0                  3
                                  1                  3
311          0                    0                 27
             1                    0                  1
Name: count, Length: 705, dtype: int64

In [138]:
# Exclude target tokens
subset = fb[~fb.distance_from_target.isin([0, 1, 2])]

# Aggregate per article: check if there is any token with Borderline/Real metaphor
agg = subset.groupby("Article_col").agg({
    "Borderline_metaphor": "max",  # 1 if any token in article is borderline
    "Real_metaphors": "max"        # 1 if any token in article is real metaphor
}).reset_index()

# Count combinations of 0/0, 0/1, 1/0, 1/1
comb_counts = agg.groupby(["Borderline_metaphor", "Real_metaphors"]).size().reset_index(name="count")

print(comb_counts)


   Borderline_metaphor  Real_metaphors  count
0                    0               0     42
1                    1               0    124
2                    1               1    146


In [145]:
before = fb[(~fb.distance_from_target.isin([0,1,2])) & (fb.distance_from_target < 0)]
after = fb[(~fb.distance_from_target.isin([0,1,2])) & (fb.distance_from_target > 2)]

In [153]:
before.Real_metaphors.mean()

0.024762325889896086

In [152]:
after.Real_metaphors.mean()

0.027545499262174127

In [154]:
before.Frame_label.value_counts()

Frame_label
_                          3089
Body_parts                  146
People                       37
Temporal_collocation         30
Experiencer_focus            29
                           ... 
Wearing                       1
Have_associated               1
Noise_makers                  1
Deserving                     1
Surrendering_possession       1
Name: count, Length: 302, dtype: int64

In [155]:
after.Frame_label.value_counts()

Frame_label
_                            2921
Experiencer_focus              53
Body_parts                     37
Emotion_directed               28
People                         27
                             ... 
Practice                        1
Quantity                        1
Make_cognitive_connection       1
Avoiding                        1
Member_of_military              1
Name: count, Length: 287, dtype: int64

In [156]:
after[after.Frame_label == "Experiencer_focus"]

Article_col  Sentence       Tokens  Borderline_metaphor  \
Article                                                                 
0       30              0         1    happiness                    0   
7       235             7         1       Lament                    0   
        241             7         1         Love                    0   
11      364            11         1      passion                    1   
13      421            13         0         love                    0   
27      846            27         1         lust                    0   
33      1035           33         3        sorry                    0   
52      1617           52         2       hatred                    0   
67      2078           67         2        sorry                    0   
70      2166           70         1        shame                    0   
77      2381           77         1         fear                    0   
        2384           77         1         lust                    0   
89      2755           89         0        pined                    0   
91      2820           91         2         love                    0   
94      2909           94         3       feared                    0   
100     3092          100         2        pride                    0   
140     4307          140         1  enthusiasm.                    0   
141     4337          141         1   enthusiasm                    0   
142     4366          142         0     patience                    0   
        4369          142         0       hunger                    0   
144     4425          144         1     jealousy                    0   
        4430          144         1    cherished                    0   
148     4546          148         3     despises                    0   
172     5281          172         1         hate                    0   
174     5332          174         0        shame                    0   
189     5795          189         1       regret                    0   
194     5946          194         1  humiliation                    0   
206     6314          206         1         love                    0   
        6318          206         1  fulfillment                    0   
        6320          206         1    happiness                    0   
214     6572          214         1        loved                    0   
225     6903          225         2         pain                    0   
243     7463          243         2         Love                    0   
        7466          243         2         hate                    0   
        7472          243         3         love                    0   
244     7490          244         1         Love                    0   
        7493          244         1         hate                    0   
        7499          244         2         love                    0   
248     7621          248         1        guilt                    0   
        7624          248         1      remorse                    0   
252     7755          252         1         love                    0   
253     7776          253         1        shame                    0   
261     8020          261         0        pride                    0   
264     8115          264         0      shyness                    0   
268     8246          268         2         LOVE                    0   
273     8392          273         4   impatience                    0   
275     8448          275         1         envy                    0   
        8454          275         1        shame                    0   
277     8530          277         2        hated                    0   
285     8766          285         1       hatred                    0   
298     9175          298         1        loves                    0   
299     9197          299         1         envy                    0   
        9203          299         1        shame                    0   

In [161]:
from scipy.stats import chi2_contingency



In [187]:
### literal analysis
range_tuples = [(30, 22), (22, 17), (17, 12), (12, 7), (7, 3), (0, -5), (-5, -10), (-10, -15), (-15, -20)]
results = []

for start, end in range_tuples:
    subset = ffb[(ffb['distance_from_target'] < start) & (ffb['distance_from_target'] >= end)]

    # per-article presence of metaphors
    has_metaphor = subset.groupby('Article_col')['Real_metaphors'].any()
    has_borderline = subset.groupby('Article_col')['Borderline_metaphor'].any()

    count_with_metaphor = has_metaphor.sum()
    count_with_borderline = has_borderline.sum()

    total = has_metaphor.shape[0]

    # rates
    rate_real = count_with_metaphor / total if total > 0 else 0
    rate_borderline = count_with_borderline / total if total > 0 else 0
    rate_total = (count_with_metaphor + count_with_borderline) / total if total > 0 else 0

    results.append({
        'interval': f"{start} to {end}",
        'count_with_metaphor': count_with_metaphor,
        'count_with_borderline': count_with_borderline,
        'total_sentences': total,
        'rate_real': rate_real,
        'rate_borderline': rate_borderline,
        'rate_total': rate_total
    })

df_results = pd.DataFrame(results)
df_results

,interval,count_with_metaphor,count_with_borderline,total_sentences,rate_real,rate_borderline,rate_total
0,30 to 22,1,1,1,1.000000,1.000000,2.000000
1,22 to 17,0,0,1,0.000000,0.000000,0.000000
2,17 to 12,8,23,77,0.103896,0.298701,0.402597
3,12 to 7,11,26,77,0.142857,0.337662,0.480519
4,7 to 3,9,17,77,0.116883,0.220779,0.337662
5,0 to -5,6,15,77,0.077922,0.194805,0.272727
6,-5 to -10,9,26,76,0.118421,0.342105,0.460526
7,-10 to -15,4,16,76,0.052632,0.210526,0.263158
8,-15 to -20,0,0,0,0.000000,0.000000,0.000000


In [196]:
# articles where the target sentence (distance = 0) has a real metaphor
excluded_articles = set(
    ffb.loc[
        (ffb['distance_from_target'] == 0) &
        (ffb['Real_metaphors'] == 1),
        'Article_col'
    ]
)
excluded_articles

range_tuples = [(30, 22), (22, 17), (17, 12), (12, 7), (7, 3), (0, -5), (-5, -10), (-10, -15), (-15, -20)]
results = []

for start, end in range_tuples:

    # interval subset
    subset = ffb[
        (ffb['distance_from_target'] < start) &
        (ffb['distance_from_target'] >= end)
    ]

    # exclude unwanted articles
    subset = subset[~subset['Article_col'].isin(excluded_articles)]

    # per-article presence of metaphors
    has_metaphor = subset.groupby('Article_col')['Real_metaphors'].any()
    has_borderline = subset.groupby('Article_col')['Borderline_metaphor'].any()

    count_with_metaphor = has_metaphor.sum()
    count_with_borderline = has_borderline.sum()

    total = has_metaphor.shape[0]

    # rates
    rate_real = count_with_metaphor / total if total > 0 else 0
    rate_borderline = count_with_borderline / total if total > 0 else 0
    rate_total = (count_with_metaphor + count_with_borderline) / total if total > 0 else 0

    results.append({
        'interval': f"{start} to {end}",
        'count_with_metaphor': count_with_metaphor,
        'count_with_borderline': count_with_borderline,
        'total_sentences': total,
        'rate_real': rate_real,
        'rate_borderline': rate_borderline,
        'rate_total': rate_total
    })

df_results = pd.DataFrame(results)
df_results


,interval,count_with_metaphor,count_with_borderline,total_sentences,rate_real,rate_borderline,rate_total
0,30 to 22,1,1,1,1.000000,1.000000,2.000000
1,22 to 17,0,0,1,0.000000,0.000000,0.000000
2,17 to 12,7,16,53,0.132075,0.301887,0.433962
3,12 to 7,5,13,53,0.094340,0.245283,0.339623
4,7 to 3,7,12,53,0.132075,0.226415,0.358491
5,0 to -5,2,9,53,0.037736,0.169811,0.207547
6,-5 to -10,7,14,52,0.134615,0.269231,0.403846
7,-10 to -15,3,11,52,0.057692,0.211538,0.269231
8,-15 to -20,0,0,0,0.000000,0.000000,0.000000


In [179]:
ffb[ffb.distance_from_target == 0].Real_metaphors.value_counts()

Real_metaphors
0    53
1    24
Name: count, dtype: int64

In [183]:
ffb[ffb.distance_from_target == 0].Borderline_metaphor.value_counts()


Borderline_metaphor
0    50
1    27
Name: count, dtype: int64

Article_col  Sentence    Tokens  Borderline_metaphor  \
Article                                                              
2       61              2         0       She                    0   
        62              2         0     never                    0   
        63              2         0    sleeps                    0   
        64              2         0      Only                    0   
        65              2         0     roams                    0   
...                   ...       ...       ...                  ...   
66      2057           66         2      Cora                    0   
        2058           66         2         ,                    0   
        2059           66         2   hearing                    0   
        2060           66         2       the                    0   
        2061           66         2  sentence                    0   

              Real_metaphors            Frame_label Token_lemma  is_target  \
Article                                                                      
2       61                 0                      _         she      False   
        62                 0              Frequency       never      False   
        63                 0                  Sleep       sleep      False   
        64                 0                      _        only      False   
        65                 0            Self_motion        roam      False   
...                      ...                    ...         ...        ...   
66      2057               0                      _        Cora      False   
        2058               0                      _           ,      False   
        2059               0  Perception_experience        hear      False   
        2060               0                      _         the      False   
        2061               0                Revenge    sentence      False   

              distance_from_target  
Article                             
2       61                     -15  
        62                     -14  
        63                     -13  
        64                     -12  
        65                     -11  
...                            ...  
66      2057                    11  
        2058                    12  
        2059                    13  
        2060                    14  
        2061                    15  

[741 rows x 9 columns]

In [182]:
real_0 = [3, 4, 6, 12, 21, 26, 28, 30, 33, 36, 38, 57, 62, 67, 79, 81, 98, 105, 107, 145, 148, 149, 158, 180, 192, 198, 200, 202, 211, 218, 232, 240, 255, 256, 261, 266, 273, 275, 277, 286, 299, 301, 308, 311]

fb[fb.Article_col.isin(real_0)]

Article_col  Sentence    Tokens  Borderline_metaphor  \
Article                                                              
3       93              3         0        at                    1   
        94              3         0       the                    0   
        95              3         0      same                    0   
        96              3         0      time                    0   
        97              3         0  requires                    0   
...                   ...       ...       ...                  ...   
311     9576          311         1         .                    0   
        9577          311         2         #                    0   
        9578          311         2    Though                    0   
        9579          311         2     still                    0   
        9580          311         2  commonly                    0   

              Real_metaphors                 Frame_label Token_lemma  \
Article                                                                
3       93                 0                           _          at   
        94                 0                           _         the   
        95                 0                Identicality        same   
        96                 0                           _        time   
        97                 0         Have_as_requirement     require   
...                      ...                         ...         ...   
311     9576               0                           _           .   
        9577               0                           _           #   
        9578               0                  Concessive      though   
        9579               0  Continued_state_of_affairs       still   
        9580               0                           _    commonly   

              is_target distance_from_target  
Article                                       
3       93        False                  -15  
        94        False                  -14  
        95        False                  -13  
        96        False                  -12  
        97        False                  -11  
...                 ...                  ...  
311     9576      False                   11  
        9577      False                   12  
        9578      False                   13  
        9579      False                   14  
        9580      False                   15  

[1355 rows x 9 columns]

In [204]:
e.to_csv("data/processed/emo.csv")
fb.to_csv("data/processed/burn.csv")
ffb.to_csv("data/processed/literal.csv")

In [1]:
### all emotion / heat data

import pandas as pd
e = pd.read_csv('data/processed/emo.csv')
range_tuples = [(30, 22), (22, 17), (17, 12), (12, 7), (7, 3), (0, -5), (-5, -10), (-10, -15), (-15, -20)]
results = []

for start, end in range_tuples:
    subset = e[(e['distance_from_target'] < start) & (e['distance_from_target'] >= end)]

    # per-article presence of metaphors
    has_metaphor = subset.groupby('Article_col')['Real_metaphors'].any()
    has_borderline = subset.groupby('Article_col')['Borderline_metaphor'].any()

    count_with_metaphor = has_metaphor.sum()
    count_with_borderline = has_borderline.sum()

    total = has_metaphor.shape[0]

    # rates
    rate_real = count_with_metaphor / total if total > 0 else 0
    rate_borderline = count_with_borderline / total if total > 0 else 0
    rate_total = (count_with_metaphor + count_with_borderline) / total if total > 0 else 0

    results.append({
        'interval': f"{start} to {end}",
        'count_with_metaphor': count_with_metaphor,
        'count_with_borderline': count_with_borderline,
        'total_sentences': total,
        'rate_real': rate_real,
        'rate_borderline': rate_borderline,
        'rate_total': rate_total
    })

df_results = pd.DataFrame(results)
df_results

KeyError: 'distance_from_target'

In [208]:
 e
 e

,Article,Sentence,Tokens,Borderline_metaphor,Real_metaphors,Frame_label,Token_lemma
0,0,0,was,0,0,_,be
1,0,0,tormented,1,1,Emotion_directed,torment
2,0,0,with,1,0,_,with
3,0,0,his,0,0,_,his
4,0,0,life,0,0,_,life
...,...,...,...,...,...,...,...
10284,333,2,of,0,0,Partitive,of
10285,333,2,your,0,0,_,your
10286,333,2,body,0,0,Body_parts,body
10287,333,2,.,0,0,_,.
